In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K

In [0]:
# Set some parameters
im_width = 480
im_height = 360
border = 5

In [0]:
ids = next(os.walk("/content/gdrive/My Drive/avc-dataset/MonuSeg/Training/TissueImages/"))[2] # list of names all images in the given path
X = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)
y = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)

In [22]:
# tqdm is used to display the progress bar
for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
    # Load images
    img = load_img("/content/gdrive/My Drive/avc-dataset/MonuSeg/Training/TissueImages/"+id_, grayscale=True)
    x_img = img_to_array(img)
    x_img = resize(x_img, (360, 480, 1), mode = 'constant', preserve_range = True)
    # Load masks
    mask_id_ = id_.split('.')[0] + '_bin_mask.png'
    mask = img_to_array(load_img("/content/gdrive/My Drive/avc-dataset/MonuSeg/Training/GroundTruth/"+mask_id_, grayscale=True))
    mask = resize(mask, (360, 480, 1), mode = 'constant', preserve_range = True)
    # Save images
    X[n] = x_img/255.0
    y[n] = mask/255.0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [0]:
# Split train and valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

In [0]:
# Define Some metrics
## Dice Score
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
## F1 Score
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [0]:
import keras
from keras.layers import Layer, ZeroPadding2D, Cropping2D
import tensorflow as tf
from tensorflow.keras import backend as K

class MaxPoolingWithIndices(keras.layers.Layer):
    def __init__(self, pool_size,strides,padding='SAME',**kwargs):
        super(MaxPoolingWithIndices, self).__init__(**kwargs)
        self.pool_size=pool_size
        self.strides=strides
        self.padding=padding
        return
    def call(self,x):
        pool_size=self.pool_size
        strides=self.strides
        if isinstance(pool_size,int):
            ps=[1,pool_size,pool_size,1]
        else:
            ps=[1,pool_size[0],pool_size[1],1]
        if isinstance(strides,int):
            st=[1,strides,strides,1]
        else:
            st=[1,strides[0],strides[1],1]
        output1,output2=tf.nn.max_pool_with_argmax(x,ps,st,self.padding)
        return [output1,output2]
    def compute_output_shape(self, input_shape):
        if isinstance(self.pool_size,int):
            output_shape=(input_shape[0],input_shape[1]//self.pool_size,input_shape[2]//self.pool_size,input_shape[3])
        else:
            output_shape=(input_shape[0],input_shape[1]//self.pool_size[0],input_shape[2]//self.pool_size[1],input_shape[3])
        return [output_shape,output_shape]


class UpSamplingWithIndices(Layer):
    def __init__(self, **kwargs):
        super(UpSamplingWithIndices, self).__init__(**kwargs)
        return
    def call(self,x):
        argmax=K.cast(K.flatten(x[1]),'int32')
        max_value=K.flatten(x[0])
        with tf.variable_scope(self.name):
            input_shape=K.shape(x[0])
            batch_size=input_shape[0]
            image_size=input_shape[1]*input_shape[2]*input_shape[3]
            output_shape=[input_shape[0],input_shape[1]*2,input_shape[2]*2,input_shape[3]]
            indices_0=K.flatten(tf.matmul(K.reshape(tf.range(batch_size),(batch_size,1)),K.ones((1,image_size),dtype='int32')))
            indices_1=argmax%(image_size*4)//(output_shape[2]*output_shape[3])
            indices_2=argmax%(output_shape[2]*output_shape[3])//output_shape[3]
            indices_3=argmax%output_shape[3]
            indices=tf.stack([indices_0,indices_1,indices_2,indices_3])
            output=tf.scatter_nd(K.transpose(indices),max_value,output_shape)
            return output
    def compute_output_shape(self, input_shape):
        return input_shape[0][0],input_shape[0][1]*2,input_shape[0][2]*2,input_shape[0][3]

def CompositeConv(inputs,num_layers,num_features):
    output=inputs
    if isinstance(num_features,int):
        for i in range(num_layers):
            output=Conv2D(num_features,(7,7),padding='same')(output)
            output=BatchNormalization(axis=3)(output)
            output=Activation('relu')(output)
        return output
    for i in range(num_layers):
        output=Conv2D(num_features[i],(7,7),padding='same')(output)
        output=BatchNormalization(axis=3)(output)
        output=Activation('relu')(output)
    return output

In [0]:
inputs=Input(shape=(360,480,1))

x = ZeroPadding2D(((12,12),(16,16)))(inputs)

x=CompositeConv(x,2,64)
x,argmax1=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

x=CompositeConv(x,2,64)
x,argmax2=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

x=CompositeConv(x,3,64)
x,argmax3=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

x=CompositeConv(x,3,64)
x,argmax4=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

x=CompositeConv(x,3,64)
x,argmax5=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

x=UpSamplingWithIndices()([x,argmax5])
x=CompositeConv(x,3,64)

x=UpSamplingWithIndices()([x,argmax4])
x=CompositeConv(x,3,64)

x=UpSamplingWithIndices()([x,argmax3])
x=CompositeConv(x,3,64)

x=UpSamplingWithIndices()([x,argmax2])
x=CompositeConv(x,2,64)

x=UpSamplingWithIndices()([x,argmax1])
x=CompositeConv(x,2,[64,1])

x=Activation('sigmoid')(x)

y=Cropping2D(((12,12),(16,16)))(x)


model=Model(inputs=inputs,outputs=y)

In [27]:
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy", dice_coef, f1])
results=model.fit(X_train, y_train, batch_size=1, epochs=8,
                    validation_data=(X_valid, y_valid))

Train on 27 samples, validate on 3 samples
Epoch 1/8
27/27 [==============================] - 35s 1s/step - loss: 0.6987 - acc: 0.5156 - dice_coef: 0.3616 - f1: 0.4114 - val_loss: 3.8594 - val_acc: 0.2256 - val_dice_coef: 0.4542 - val_f1: 0.4649
Epoch 2/8
27/27 [==============================] - 25s 930ms/step - loss: 0.6658 - acc: 0.5827 - dice_coef: 0.4013 - f1: 0.5829 - val_loss: 0.7657 - val_acc: 0.3282 - val_dice_coef: 0.4319 - val_f1: 0.4826
Epoch 3/8
27/27 [==============================] - 25s 927ms/step - loss: 0.6582 - acc: 0.5903 - dice_coef: 0.4137 - f1: 0.6191 - val_loss: 0.7215 - val_acc: 0.3075 - val_dice_coef: 0.4322 - val_f1: 0.4779
Epoch 4/8
27/27 [==============================] - 25s 928ms/step - loss: 0.6554 - acc: 0.5925 - dice_coef: 0.4150 - f1: 0.6249 - val_loss: 0.7235 - val_acc: 0.3468 - val_dice_coef: 0.4329 - val_f1: 0.4959
Epoch 5/8
27/27 [==============================] - 25s 929ms/step - loss: 0.6546 - acc: 0.5914 - dice_coef: 0.4171 - f1: 0.6234 - val_lo

In [28]:
# Evaluate on validation set (this must be equals to the best log_loss)
loss, accuracy, f1_score, dice_score = model.evaluate(X_valid, y_valid, verbose=1)

print("Loss:", loss)
print("Accuracy:", accuracy)
print("F1_score:", f1_score)
print("Dice_score:", dice_score)

3/3 [==============================] - 1s 339ms/step
Loss: 0.6790745854377747
Accuracy: 0.4351601004600525
F1_score: 0.4169818162918091
Dice_score: 0.5396543145179749
